In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy.linalg as spla
import numpy.linalg as npla
import seaborn as sns
from itertools import groupby
from matplotlib.colors import LogNorm
from tqdm.notebook import tqdm
import csv

In [2]:
from model_pvp import model_pvp
from generate_distribution import generate_distribution
from make_prob_matrix import make_prob_matrix
from get_fundamental_matrix import get_fundamental_matrix
from get_mean_time import get_mean_time
from get_border_cases import get_border_cases

In [3]:
from digitalize_states import digitalize_states
import get_games_strategy

In [4]:
N = 16

counts_pvp = np.load('../data/qr_counts_pvp.npy')
counts_pvp_400 = np.load('../data/qr_counts_pvp_400.npy')
counts_pve_border = np.load('../data/qr_counts_pve_border.npy')
counts_pve_center = np.load('../data/qr_counts_pve_center.npy')

qr_pve_border = np.load('../data/qr_pve_border.npy')
qr_pve_center = np.load('../data/qr_pve_center.npy')
qr_pvp = np.load('../data/qr_pvp.npy')
qr_pvp_from_100 = np.load('../data/qr_pvp_from_100.npy')
qr_pvp_400 = np.load('../data/qr_pvp_400.npy')

# PvE Center
strategy_pve_center = np.load('../data/strategy_pve_center.npy')

# PvE Border
strategy_pve_border = np.load('../data/strategy_pve_border.npy')

# Random
strategy_random = np.ones ((N + 1, N + 1), dtype=np.float64) * 0.5

# PvP from 100 Border
strategy_pvp_from_100_border = np.load('../data/strategy_pvp_from_100_border.npy')

# PvP from 100 Border
strategy_pvp_from_100_center = np.load('../data/strategy_pvp_from_100_center.npy')

# PvP Border
strategy_pvp_border = np.load('../data/strategy_pvp_border.npy')

# PvP Center
strategy_pvp_center = np.load('../data/strategy_pvp_center.npy')

# PvP 400+ Border
strategy_pvp_400_border = np.load('../data/strategy_pvp_400_border.npy')

# PvP 400+ Center
strategy_pvp_400_center = np.load('../data/strategy_pvp_400_center.npy')

# PvE Border Optimal
strategy_pve_border_optimal = np.zeros ((N + 1, N + 1), dtype=np.float64)
qr_pve_border_optimal, probabilities = make_prob_matrix(N, strategy_random, strategy_pve_border_optimal)

# PvE Center Optimal
strategy_pve_center_optimal = np.zeros((N + 1, N + 1), dtype=np.float64)
strategy_pve_center_optimal = \
    np.diag(np.ones(N + 1) * 0.5) + \
    np.diag(np.ones(N), -1) + \
    np.diag(np.zeros(N), +1)
strategy_pve_center_optimal = strategy_pve_center_optimal.T
qr_pve_center_optimal, probabilities = make_prob_matrix(N, strategy_pve_center_optimal, strategy_random)

# PvE Center Optimal 2d 
strategy_pve_center_optimal_2d = np.zeros((N + 1, N + 1), dtype=np.float64)
strategy_pve_center_optimal_2d[1:N, 1] = 1
strategy_pve_center_optimal_2d[N - 1, 1:N] = 1
strategy_pve_center_optimal_2d = strategy_pve_center_optimal_2d.T
qr_pve_center_optimal_2d, probabilities = make_prob_matrix(N, strategy_pve_center_optimal_2d, strategy_random)

# BvB
qr_bvb, probabilities = make_prob_matrix(N, strategy_random, strategy_random)

qr_pve_border_pure, probabilities = make_prob_matrix(N, strategy_random, strategy_pve_border)
qr_pve_center_pure, probabilities = make_prob_matrix(N, strategy_pve_center, strategy_random)
qr_pvp_pure, probabilities = make_prob_matrix(N, strategy_pvp_center, strategy_pvp_border)
qr_pvp_400_pure, probabilities = make_prob_matrix(N, strategy_pvp_400_center, strategy_pvp_400_border)

In [5]:
from get_prob_matrix import get_frequencies

In [6]:
qrs = {
    'pve_border': qr_pve_border,
    'pve_center': qr_pve_center,
    'bvb': qr_bvb,
    'pve_border_pure': qr_pve_border_pure,
    'pve_center_pure': qr_pve_center_pure,
    'pvp': qr_pvp,
    'pvp_from_100': qr_pvp_from_100,
    'pvp_pure': qr_pvp_pure,
    'pvp_400': qr_pvp_400,
    'pvp_400_pure': qr_pvp_400_pure,
    'pve_border_optimal': qr_pve_border_optimal,
    'pve_center_optimal': qr_pve_center_optimal,
    'pve_center_optimal_2d': qr_pve_center_optimal_2d,
}

In [7]:
def remove_absorption_states(qr, N):
    new_matrix = qr.copy()
    border_states = get_border_cases(N)
    
    new_matrix = np.delete(new_matrix, border_states, axis=0)
    new_matrix = np.delete(new_matrix, border_states, axis=1)
    
    return new_matrix

In [8]:
def get_max_eigenvectors_h(qr):
    e_values, e_vec_r = npla.eigh(qr) 
    
    max_ev_index = np.where(np.isclose(abs(e_values), abs(max(e_values, key=abs))))[0][0]
    
    max_right_vec = e_vec_r[:, max_ev_index]
    
    return e_values, e_vec_r, max_right_vec, max_ev_index

In [9]:
'''
Returns a vector, which has a non-zero value in center of grid.
'''
def get_nonzero_vector(vectors):
    for v in vectors.T:
        if (v[7 * 15 + 7] != 0):
            return v
    return vectors.T[0]

In [10]:
def get_max_eigenvectors(qr, index_max=0, index_min=-1):
    e_values, e_vec_l, e_vec_r = spla.eig(qr, left=True, right=True) 
    
    # Make eigenvalues real
    e_values = np.real(e_values)
    
    # Make an array of indices for eigenvalues
    indices = range(0, len(e_values))  
    
    # Sort eigenvalues array from Max to Min
    sorted_e_values, indices = zip(*sorted(zip(e_values, indices), reverse=True))
    indices = list(indices)
    
    
    # Get unique eigenvalues and their multiplicity. Array is sorted from Min to Max
    unique_e_vals, counts = np.unique(sorted_e_values, return_counts=True)
    
    # Reverse counts to match e_values order
    counts = counts[::-1]
    
    # We need to get all vectors corresponding to Max eigenvalue
    # Max eigenvalue has index of indices[0]. There are counts[0] of vectors
    # corresponing to that eigenvalue. 
    # Supposedly, these vectors are first in indices[0 : counts[0]] places for positive eigenvalue.
    # For negative value, it's indicies[-counts[-1] : ]
    left_vectors_for_max_positive_evalue = e_vec_l[:, indices[0 : counts[0]]]
    right_vectors_for_max_positive_evalue = e_vec_r[:, indices[0 : counts[0]]]
    
    left_vectors_for_min_negative_evalue = e_vec_l[:, indices[-counts[-1] : ]] 
    right_vectors_for_min_negative_evalue = e_vec_r[:, indices[-counts[-1] : ]]
    
   
    # Get vectors which middle value is not zero
    # Shape of input matrix is (225 x K), where K is the eigenvalue multiplicity
    max_left_vec_positive = get_nonzero_vector(left_vectors_for_max_positive_evalue)
    max_right_vec_positive = get_nonzero_vector(right_vectors_for_max_positive_evalue)
    
    max_left_vec_negative = get_nonzero_vector(left_vectors_for_min_negative_evalue)
    max_right_vec_negative = get_nonzero_vector(right_vectors_for_min_negative_evalue)
    
    # Normalize the vectors by sum
    max_left_vec_positive = max_left_vec_positive / np.sum(np.abs(max_left_vec_positive))
    max_right_vec_positive = max_right_vec_positive / np.sum(np.abs(max_right_vec_positive))
    max_left_vec_negative = max_left_vec_negative / np.sum(np.abs(max_left_vec_negative))
    max_right_vec_negative = max_right_vec_negative / np.sum(np.abs(max_right_vec_negative))
    
    return sorted_e_values, indices, e_values, e_vec_l, e_vec_r, \
    max_left_vec_positive, \
    max_right_vec_positive, \
    max_left_vec_negative, \
    max_right_vec_negative 

In [11]:
def visualise_eigenvalues(e_values, case):
    plt.clf()
    unique_e_vals, counts = np.unique(e_values, return_counts=True)
    
    fig, ax = plt.subplots(1, figsize=(16, 9), dpi=300)
        
    xs_for_axis = unique_e_vals
    ys_for_axis = np.zeros(shape=(len(xs_for_axis)))
        
    ax.set_title("Eigenvalues for " + case)
    ax.set_xlabel("Eigenvalue")
    ax.set_ylabel("Eigenvalue multiplicity")
    ax.plot(xs_for_axis, ys_for_axis, marker='x')
    
    for i in range(len(unique_e_vals)):
        x = unique_e_vals[i]
        y_max = counts[i]
        
        ys = np.arange(start=0, stop=y_max, step=0.01)
        xs = [x] * len(ys)
        ax.plot(xs, ys)
        ax.set_yticks(np.arange(start=0, stop=y_max + 1, step=1))
        plt.close(fig)
    
    fig.savefig("../Eigenvectors/" + case + '/' + "eigenvalues.png", transparent=False, \
                facecolor='white', edgecolor='white')
    
    return unique_e_vals, counts

In [12]:
def visualise_eigenvector(vector, N, case, lr, evalue, save=True):    
    fig, ax = plt.subplots(1, figsize=(12, 9), dpi=300)
    
    ax.set_yticks(np.arange(start=0, stop=N, step=2))
    ax.set_xticks(np.arange(start=0, stop=N, step=2))
    ax.xaxis.tick_top()
    
    grid = np.zeros(shape=(N, N))
    for i in range(N):
        for j in range(N):
            grid[i, j] = np.real(vector[i * N + j])
            
    ax.set_xlim((-0.5, N-0.5))
    ax.set_ylim((-0.5, N-0.5)) 
    t = ax.imshow(grid)
    plt.close()
    ax.invert_yaxis()
    fig.colorbar(t)
    
    ax.set_title(qr + " Eigenvector \n" + lr + " " + str(evalue))
    if save:
        fig.savefig("../Eigenvectors/" + case + '/' + "eigenvector_" + lr + ".png", transparent=False, \
                    facecolor='white', edgecolor='white')
    else:
        fig.show()

In [13]:
def visualise_qr(qr, case):
    fig, ax = plt.subplots(1, dpi=300)
    ax.set_title("QR matrix for " + case)
    
    ax.set_yticks(np.arange(start=0, stop=qr.shape[0], step=20))
    ax.set_xticks(np.arange(start=0, stop=qr.shape[1], step=20))
    ax.xaxis.tick_top()
    ax.invert_yaxis()
    t = ax.imshow(qr)
    plt.close()
    fig.colorbar(t)
    fig.savefig("../Eigenvectors/" + case + '/' + "qr.png", transparent=False, \
                facecolor='white', edgecolor='white')

In [14]:
from pathlib import Path
def make_directory(case):
    p = Path("../Eigenvectors/" + case + '/')
    p.mkdir(parents=True, exist_ok=True)

In [15]:
import csv
def save_evalues(unique_evals, counts, case, vl_pos, vr_pos, vl_neg, vr_neg):
    
    unique_evals = np.real(unique_evals)
    
    unique_evals, counts = zip(*sorted(zip(unique_evals, counts), reverse=True))
        
    unique_evals = list(unique_evals)
    counts = list(counts)
    path = '../Eigenvectors/' + case + '/' + 'values.txt'

    
    with open(path, 'w') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        
        out_file.write('l_2 / l_1\n')
        out_file.write(str(unique_evals[1] / unique_evals[0]))
        out_file.write('\n')
        
        
        out_file.write('lambdas\n')
        
        [out_file.write('{:.15f} '.format(x)) for x in unique_evals]
        out_file.write('\n')
        [out_file.write('{0:17d} '.format(c)) for c in counts]
        out_file.write('\n')
        
        out_file.write('max left eigenvector for positive lambda\n')
        tsv_writer.writerow(vl_pos)
        out_file.write("max right eigenvector for positive lambda\n")
        tsv_writer.writerow(vr_pos)

        out_file.write("max left eigenvector for negative lambda\n")
        tsv_writer.writerow(vl_neg)
        out_file.write("max right eigenvector for negative lambda\n")
        tsv_writer.writerow(vr_neg)
        
        out_file.write('max left * right eigenvector for positive lambda\n')
        tsv_writer.writerow(vl_pos * vr_pos / np.sum(np.abs(vl_pos * vr_pos)))
        out_file.write("max left * right eigenvector for negative lambda\n")
        tsv_writer.writerow(vl_neg * vr_neg / np.sum(np.abs(vl_neg * vr_neg)))

In [16]:
for qr in tqdm(qrs):
    make_directory(qr)
    matrix = qrs[qr]
    mx = remove_absorption_states(matrix, N)
    sorted_e_values, indices, e_values, e_vec_l, e_vec_r, vl_pos, vr_pos, vl_neg, vr_neg = get_max_eigenvectors(mx)
    
    unique_evals, counts = visualise_eigenvalues(sorted_e_values, qr)
    
    # Visualise left eigenvector for max positive eigenvalue
    vl_pos = vl_pos if (vl_pos[7 * 15 + 7] > 0) else -vl_pos
    visualise_eigenvector(vl_pos, N - 1, qr, \
                              'left for positive eigenvalue', sorted_e_values[0])

    vr_pos = vr_pos if (vr_pos[7 * 15 + 7] > 0) else -vr_pos
    # Visualise right eigenvector for max positive eigenvalue
    visualise_eigenvector(vr_pos, N - 1, qr, \
                              'right for positive eigenvalue', sorted_e_values[0])

    vl_neg = vl_neg if (vl_neg[7 * 15 + 7] > 0) else -vl_neg
    # Visualise left eigenvector for min negative eigenvalue
    visualise_eigenvector(vl_neg, N - 1, qr, \
                              'left for negative eigenvalue', sorted_e_values[-1])

    vr_neg = vr_neg if (vr_neg[7 * 15 + 7] > 0) else -vr_neg
    # Visualise right eigenvector for min negative eigenvalue
    visualise_eigenvector(vr_neg, N - 1, qr, \
                              'right for negative eigenvalue', sorted_e_values[-1])

    # Visualise multiplication of left and right vectors for max positive eigenvalue
    visualise_eigenvector(vl_pos * vr_pos / np.sum(np.abs(vl_pos * vr_pos)), N - 1, qr,  'Left x right for positive eigenvalue', sorted_e_values[0])

    # Visualise multiplication of left and right vectors for min negative eigenvalue
    visualise_eigenvector(vl_neg * vr_neg / np.sum(np.abs(vl_neg * vr_neg)), N - 1, qr, 'Left x right for negative eigenvalue', sorted_e_values[-1])

    visualise_qr(mx, qr)

    save_evalues(unique_evals, counts, qr, vl_pos, vr_pos, vl_neg, vr_neg)

    

  0%|          | 0/13 [00:00<?, ?it/s]

<Figure size 432x288 with 0 Axes>

In [38]:
for elem in e_vec_l[:, indices[1]]:
    print(elem)

(0.07056108460009786+0j)
(0.07319379577054548+0j)
(0.05253074753869069+0j)
(0.03161539452142214+0j)
(0.0204248575734793+0j)
(0.012480496780464718+0j)
(0.008662915326724434+0j)
(0.004147327239178674+0j)
(0.0025555375708141296+0j)
(0.0016645038475598826+0j)
(0.0013324028252355853+0j)
(0.0006929179084158813+0j)
(0.00023779111396688882+0j)
(-0.00028298295683671976+0j)
(-0.0006866160418174292+0j)
(0.0822440152997639+0j)
(0.12103863296273867+0j)
(0.12074502830113581+0j)
(0.08386513865072116+0j)
(0.05034733004335467+0j)
(0.027129885790978997+0j)
(0.01786281924172165+0j)
(0.01257103975021366+0j)
(0.007960192715297845+0j)
(0.006031180719871437+0j)
(0.003395073493831666+0j)
(0.0006751093591372622+0j)
(0.00034751953573906684+0j)
(-0.002748686375724266+0j)
(-0.0023036693793346977+0j)
(0.061686619645511906+0j)
(0.12697925213111427+0j)
(0.1588284769001154+0j)
(0.12539089128940176+0j)
(0.0817611600503452+0j)
(0.04977835071668535+0j)
(0.0295905265969934+0j)
(0.01865148539640462+0j)
(0.0119770523453243

In [39]:
for elem in e_vec_r[:, indices[1]]:
    print(elem)

(0.010218725692338799+0j)
(0.01930946061001276+0j)
(0.02668174150287638+0j)
(0.03685360864192258+0j)
(0.044276066726305116+0j)
(0.05001103566086066+0j)
(0.05189864366056291+0j)
(0.061466018510312886+0j)
(0.05489823507635168+0j)
(0.051455495023377866+0j)
(0.058134264698127054+0j)
(0.06169870423139865+0j)
(0.06352009421881696+0j)
(-0.04665586822748724+0j)
(-0.045318046986997235+0j)
(0.018223536249576917+0j)
(0.036077511287024584+0j)
(0.04996938937858489+0j)
(0.060420545616454976+0j)
(0.06962894615395358+0j)
(0.07527602696050602+0j)
(0.08000814853232384+0j)
(0.08332211620481353+0j)
(0.07968607347736109+0j)
(0.0705792598217012+0j)
(0.058863425965319606+0j)
(0.0035607725572774033+0j)
(-0.05336372482836722+0j)
(-0.06446815078865734+0j)
(-0.04401858674458755+0j)
(0.02874706523302805+0j)
(0.04958700057153068+0j)
(0.06487227682719184+0j)
(0.07654803316791182+0j)
(0.0859211000342031+0j)
(0.09071554524585884+0j)
(0.09309838723869458+0j)
(0.09106764296666596+0j)
(0.08423511987477261+0j)
(0.0716569